# 1 - Rule-based NER

In [1]:
!pip install PyMuPDF geotext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 63.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.8 MB/s eta 0:00:00:00:01


In [2]:
!pip install PyPDF2
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00ta 0:00:01


In [3]:
# Import libraries
import fitz  # PyMuPDF for PDF text extraction
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings("ignore")
import re
from geotext import GeoText
import pandas as pd
import time

In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
!ls /kaggle/input/

corpus	resume4


In [6]:
warnings.filterwarnings("ignore")
# convert the pdf to dataframe
import fitz
def pdf_to_text(document):
    doc = fitz.open(document)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [7]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords,wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import warnings
warnings.filterwarnings("ignore")

def getWordnetPos(words):
    tag=pos_tag([words])[0][1][0].upper()
    tag_dict={"J":wordnet.ADJ,
              "N":wordnet.NOUN,
              "V":wordnet.VERB,
              "R":wordnet.ADV
             }
    return tag_dict.get(tag,wordnet.NOUN)

def cv_preprocessing(cv_data):
    #Tokenization
    tokenized_text=word_tokenize(cv_data)

    #Remove stopwords
    stop_words = set(stopwords.words('english'))
    filter_text=[]
    for token in tokenized_text:
        if token not in stop_words:
            filter_text.append(token)

    #POS and lemmatize
    lemmatizer = WordNetLemmatizer()
    lemmatizeResults=[lemmatizer.lemmatize(token,getWordnetPos(token)) for token in filter_text]
    return ' '.join(lemmatizeResults)

## 1.0 Rule based Functions

In [8]:
import re
from geotext import GeoText
import pandas as pd

def get_candidate_name(resume_text):
    name_regex = r'(?P<last_name>[A-Z][a-z]+)\s(?P<first_name>(?:[A-Z][a-z]+\.?\s?)+)|(?P<name>(?:[A-Z][a-z]+\.?\s?)+)'
    matches = re.findall(name_regex, resume_text)
    for match in matches:
        if match[0] != "":
            if match[0] not in {"He", "His", "She", "Her", "It", "They", "Their", "The", "A", "An"}:
                return(match[0] + " " +  match[1])
            
def get_phone_number(resume_text):
    pattern = r"\(\d{3}\) \d{3}-\d{4}|\d{3}-\d{3}-\d{4}"
    matches = re.findall(pattern, resume_text)
    #print(matches)
    return set(matches)


def get_email(resume_text):
    pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b"
    matches = re.findall(pattern, resume_text)
    # print(matches)
    return set(matches)


def get_qualifications(resume_text):
    # Define regular expressions for different degree types
    bachelors_regex = r"(Bachelor.*)"
    masters_regex = r"(Master.*)"
    phd_regex = r"(Doctor.*)"
    diploma_regex = r"(Diploma.*)"

    # Search for degree types in the resume text
    bachelors_match = re.search(bachelors_regex, resume_text, re.IGNORECASE)
    masters_match = re.search(masters_regex, resume_text, re.IGNORECASE)
    phd_match = re.search(phd_regex, resume_text, re.IGNORECASE)
    diploma_match = re.search(diploma_regex, resume_text, re.IGNORECASE)

    # Determine the highest education level
    if phd_match:
        education_level = "PhD"
    elif masters_match:
        education_level = "Master's degree"
    elif diploma_match and bachelors_match:
        education_level = "Bachelor's degree and Diploma"
    elif bachelors_match:
        education_level = "Bachelor's degree"
    elif diploma_match:
        education_level = "Diploma"
    else:
        education_level = "Unknown"

    return education_level


def get_graduation_year(resume_text):
    # Define a regex pattern to match graduation year
    pattern = r"(?<!\d)(20\d{2}|19\d{2})(?!\d)"
    
    # Use regex to find all graduation year matches in the resume text
    matches = re.findall(pattern, resume_text)
    
    # Return a list of unique graduation year matches
    unique_matches = set(matches)
    graduation_years = [int(match) for match in unique_matches]
    return set(graduation_years)


def get_location(resume_text):
    places = GeoText(resume_text).cities
    return set(places)


def get_skills(resume_text):
    corpus_file = open("/kaggle/input/resume4/skill_set.txt", "r")
    corpus = corpus_file.read()
    corpus = corpus.split('\n')
    return [keyword for keyword in corpus if keyword in resume_text]


def get_university_name(resume_text):
    pattern = r"(?<!\w)[A-Z][a-z']*(?:\s+[A-Z][a-z']*)*(?!\w)(?:\s+(?:College|University))"
    matches = re.findall(pattern, resume_text)
    return list(set(matches))


def get_company(resume_text):
    corpus_file = open("/kaggle/input/resume4/company.txt", "r")
    corpus = corpus_file.read()
    corpus = corpus.split('\n')
    return [keyword for keyword in corpus if keyword in resume_text]


def create_ngrams(text, n):
    # clean the text by removing non-alphanumeric characters and converting to lowercase
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    # create n-grams
    ngrams = []
    words = cleaned_text.split()
    for i in range(len(words) - n + 1):
        ngrams.append(' '.join(words[i:i+n]))
    return ngrams

def get_occupation(resume_text):
    # load job title corpus
    with open('/kaggle/input/resume4/job-titles.txt') as f:
        job_titles = f.read().splitlines()

    # create n-grams of the resume text
    resume_ngrams = set(create_ngrams(resume_text, 2) + create_ngrams(resume_text, 3))
    # match n-grams against job titles
    matching_job_titles = []
    for title in job_titles:
        if any(ngram == title.lower() for ngram in resume_ngrams):
            matching_job_titles.append(title)
    return matching_job_titles

In [9]:
import time

def rule_based_ner(file_name, resume_text):
    start_time = time.time()
    
    # get each entites
    name = get_candidate_name(resume_text)
    phone_num = get_phone_number(resume_text)
    email = get_email(resume_text)
    qualifications = get_qualifications(resume_text)
    graduated_year = get_graduation_year(resume_text)
    location = get_location(resume_text)
    skills = get_skills(resume_text)
    university = get_university_name(resume_text)
    company = get_company(resume_text)
    designation = get_occupation(resume_text)
    
    
    # print out the result
    print("=================================== RESULT OF RULE-BASED NER ===================================")
    print("Name: " , name)
    print("\nPhone Number: " , phone_num)
    print("\nEmail: " , email)
    print("\nQualifications: " , qualifications)
    print("\nGraduation Year: " , graduated_year)
    print("\nLocation: " , location)
    print("\nSkills: " , skills)
    print("\nUniversity: " , university)
    print("\nCompany: " , company)
    print("\nDesignation: " , designation)
    print("======================================== END OF RB NER ========================================")
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Execution time: {:.2f} seconds".format(elapsed_time))
    # return the values
    return file_name, name,phone_num,email,qualifications,graduated_year,location, skills,university, company, designation
    

In [10]:

def get_candidate_name(resume_text):
    name_regex = r'(?P<last_name>[A-Z][a-z]+)\s(?P<first_name>(?:[A-Z][a-z]+\.?\s?)+)|(?P<name>(?:[A-Z][a-z]+\.?\s?)+)|(?P<all_caps_name>[A-Z]+)'
    matches = re.findall(name_regex, resume_text)
    for match in matches:
        if match[0] != "":
            if match[0] not in {"He", "His", "She", "Her", "It", "They", "Their", "The", "A", "An"}:
                return(match[0] + " " +  match[1])

## 2 - Training set

### Create dataframe to store the result 

In [11]:
# Create DataFrames
train_df = pd.DataFrame(columns=['file_name', 'name', 'phone_num', 'email', 'qualifications', 'graduated_year', 'location', 'skills', 'university', 'company', 'designation'])
df = pd.DataFrame(columns=['file_name', 'name', 'phone_num', 'email', 'qualifications', 'graduated_year', 'location', 'skills', 'university', 'company', 'designation'])
df2 = pd.DataFrame(columns=['file_name', 'name', 'phone_num', 'email', 'qualifications', 'graduated_year', 'location', 'skills', 'university', 'company', 'designation'])

In [12]:
# Process training resumes (R1.pdf to R5.pdf)
resume_files = [f"/kaggle/input/resume4/R{i}.pdf" for i in range(1, 6)]
for i, resume_file in enumerate(resume_files):
    train_df.loc[i] = rule_based_ner(resume_file, pdf_to_text(resume_file))

=================================== RESULT OF RULE-BASED NER ===================================
Name:  Abiral Pandey 

Phone Number:  {'940-242-3303'}

Email:  {'abiral.pandey88@gmail.com'}

Qualifications:  Bachelor's degree

Graduation Year:  {2016, 2012, 2013, 2014, 2015}

Location:  {'Texas', 'University', 'Woonsocket', 'Mongo', 'Spring', 'Denton'}

Skills:  ['Java', 'SQL', 'JavaScript', 'MongoDB', 'Git']

University:  []

Company:  ['HP', 'Oracle', 'Toll Brothers', 'IBM', 'Visa', '']

Designation:  ['j2ee developer', 'java developer', 'junior java developer', 'software engineer']
======================================== END OF RB NER ========================================
Execution time: 5.97 seconds
=================================== RESULT OF RULE-BASED NER ===================================
Name:  Suresh Basetti
Mobile

Phone Number:  {'925-900-3354'}

Email:  {'sureshkumar.basetti@gmail.com'}

Qualifications:  Master's degree

Graduation Year:  {2017, 2001, 2002, 2004, 20

In [13]:
train_df

,file_name,name,phone_num,email,qualifications,graduated_year,location,skills,university,company,designation
0,/kaggle/input/resume4/R1.pdf,Abiral Pandey,{940-242-3303},{abiral.pandey88@gmail.com},Bachelor's degree,"{2016, 2012, 2013, 2014, 2015}","{Texas, University, Woonsocket, Mongo, Spring,...","[Java, SQL, JavaScript, MongoDB, Git]",[],"[HP, Oracle, Toll Brothers, IBM, Visa, ]","[j2ee developer, java developer, junior java d..."
1,/kaggle/input/resume4/R2.pdf,Suresh Basetti\nMobile,{925-900-3354},{sureshkumar.basetti@gmail.com},Master's degree,"{2017, 2001, 2002, 2004, 2005, 2009, 2010, 2014}","{Mobile, Vista}","[Java, SQL, JavaScript, Spring Boot, Git]",[Osmania University],"[HP, Oracle, Broadcom, TCS, ]","[agile project manager, delivery manager, hado..."
2,/kaggle/input/resume4/R3.pdf,Chandler Robert Durairaj Joshua\nMobile,{515-257-3838},{chandler.neel@gmail.com},Master's degree,"{2016, 2000, 2003, 2005, 2008, 2009, 2010, 201...","{Baltimore, Mobile, University, Madurai, Chenn...",[SQL],"[Madurai Kamaraj University, Johns Hopkins Med...","[HP, Microsoft, ]","[business analyst, business development, custo..."
3,/kaggle/input/resume4/R4.pdf,Deepika Chintalapati\nMobile,{518-805-9569},{chintalapatideepika@gmail.com},Master's degree,"{2016, 2017, 2000, 2003, 2005, 2008, 2009, 201...","{Mobile, Golden Gate, Trenton, Albany, Hyderabad}",[SQL],[],"[HP, Oracle, ServiceNow, IBM, Trent, ITI, ]","[change management, database administrator, en..."
4,/kaggle/input/resume4/R5.pdf,Mounika Kalmekolan\nMobile,{414-909-0756},{Mounika10200@gmail.com},Master's degree,{},"{Plano, Mobile}","[Java, SQL]",[],"[HP, Oracle, ]",[business analyst]


## 3 - Testing on Resume and validate the accuracy

Take 2-3 resume to test on the result, to see whether the rule-based function is performing well and is able to extract the info.

### 3.1 Without Preprocessed

In [14]:
# Process test resumes without preprocessing (R6.pdf to R10.pdf)
test_files = [f"/kaggle/input/resume4/R{i}.pdf" for i in range(6, 11)]
for i, test_file in enumerate(test_files):
    df.loc[i] = rule_based_ner(test_file, pdf_to_text(test_file))

=================================== RESULT OF RULE-BASED NER ===================================
Name:  Gopi Mobile

Phone Number:  {'732-630-7623'}

Email:  {'gopijavafullstack@gmail.com'}

Qualifications:  Bachelor's degree

Graduation Year:  {2016, 2017, 2000, 2010, 2011, 2012, 2014, 2015}

Location:  {'Mobile', 'Los Angeles', 'Frisco', 'Ajax', 'Mongo', 'Hyderabad', 'Spring', 'New York'}

Skills:  ['Java', 'SQL', 'JavaScript', 'MongoDB', 'Spring Boot', 'Git']

University:  []

Company:  ['State Street', 'Oracle', 'Amazon', 'IBM', 'Pool', 'NCC', 'ICICI Bank', '']

Designation:  ['business objects', 'data management', 'full stack developer', 'java developer', 'junior java developer', 'patient care']
======================================== END OF RB NER ========================================
Execution time: 14.61 seconds
=================================== RESULT OF RULE-BASED NER ===================================
Name:  Vora Phone

Phone Number:  {'201-532-6397'}

Email:  {'kashy

In [15]:
df

,file_name,name,phone_num,email,qualifications,graduated_year,location,skills,university,company,designation
0,/kaggle/input/resume4/R6.pdf,Gopi Mobile,{732-630-7623},{gopijavafullstack@gmail.com},Bachelor's degree,"{2016, 2017, 2000, 2010, 2011, 2012, 2014, 2015}","{Mobile, Los Angeles, Frisco, Ajax, Mongo, Hyd...","[Java, SQL, JavaScript, MongoDB, Spring Boot, ...",[],"[State Street, Oracle, Amazon, IBM, Pool, NCC,...","[business objects, data management, full stack..."
1,/kaggle/input/resume4/R7.pdf,Vora Phone,{201-532-6397},{kashyapkvora@gmail.com},Master's degree,"{2016, 2006, 2008, 2009, 2011, 2013, 2014, 2015}","{Baltimore, Mumbai, University, Smithfield, Wa...","[Python, Java, SQL, JavaScript]",[Johns Hopkins University],"[HP, Oracle, ServiceNow, Splunk, IBM, ]","[business analyst, data scientist, product man..."
2,/kaggle/input/resume4/R8.pdf,Madhuri Gogula\n,{516-358-1998},{mike@sqldatasolutionsinc.com},Bachelor's degree,"{2016, 2017, 1998, 2005, 2008, 2010, 2011, 201...","{Madison, Phoenix, McLean, Pittsburgh, Hyderab...","[Java, SQL, JavaScript, Spring Boot]",[],"[Oracle, IBM, American Express, Intel, Freddie...",[java developer]
3,/kaggle/input/resume4/R9.pdf,Technical Project Manager,"{646-361-0131, 646-361-1031}",{JNandini1324@gmail.com},Master's degree,"{2016, 2017, 2007, 2009, 2010, 2011, 2012, 201...",{},"[Java, SQL]",[],"[Oracle, Cisco Systems, Tech Mahindra, Symphon...","[business analyst, client delivery manager, cl..."
4,/kaggle/input/resume4/R10.pdf,Rama Krishna\nFull Stack Java Developer\n,{(516) 308-1026},{ramglobal5111@gmail.com},Bachelor's degree,"{2016, 2009, 2012, 2013, 2014, 2015}","{Portland, Allen, Mongo, Spring, Durham}","[Java, SQL, JavaScript, MongoDB, Spring Boot, ...",[Degree\nKoneru Lakshmaiah College],"[Oracle, ]","[data management, java developer, software dev..."


### 3.2 With Preprocessing

In [16]:
# Process test resumes with preprocessing (R6.pdf to R10.pdf)
test_files = [f"/kaggle/input/resume4/R{i}.pdf" for i in range(6, 11)]
for i, test_file in enumerate(test_files):
    df2.loc[i] = rule_based_ner(test_file, pdf_to_text(test_file))

=================================== RESULT OF RULE-BASED NER ===================================
Name:  Gopi Mobile

Phone Number:  {'732-630-7623'}

Email:  {'gopijavafullstack@gmail.com'}

Qualifications:  Bachelor's degree

Graduation Year:  {2016, 2017, 2000, 2010, 2011, 2012, 2014, 2015}

Location:  {'Mobile', 'Los Angeles', 'Frisco', 'Ajax', 'Mongo', 'Hyderabad', 'Spring', 'New York'}

Skills:  ['Java', 'SQL', 'JavaScript', 'MongoDB', 'Spring Boot', 'Git']

University:  []

Company:  ['State Street', 'Oracle', 'Amazon', 'IBM', 'Pool', 'NCC', 'ICICI Bank', '']

Designation:  ['business objects', 'data management', 'full stack developer', 'java developer', 'junior java developer', 'patient care']
======================================== END OF RB NER ========================================
Execution time: 14.00 seconds
=================================== RESULT OF RULE-BASED NER ===================================
Name:  Vora Phone

Phone Number:  {'201-532-6397'}

Email:  {'kashy

In [17]:
df2

,file_name,name,phone_num,email,qualifications,graduated_year,location,skills,university,company,designation
0,/kaggle/input/resume4/R6.pdf,Gopi Mobile,{732-630-7623},{gopijavafullstack@gmail.com},Bachelor's degree,"{2016, 2017, 2000, 2010, 2011, 2012, 2014, 2015}","{Mobile, Los Angeles, Frisco, Ajax, Mongo, Hyd...","[Java, SQL, JavaScript, MongoDB, Spring Boot, ...",[],"[State Street, Oracle, Amazon, IBM, Pool, NCC,...","[business objects, data management, full stack..."
1,/kaggle/input/resume4/R7.pdf,Vora Phone,{201-532-6397},{kashyapkvora@gmail.com},Master's degree,"{2016, 2006, 2008, 2009, 2011, 2013, 2014, 2015}","{Baltimore, Mumbai, University, Smithfield, Wa...","[Python, Java, SQL, JavaScript]",[Johns Hopkins University],"[HP, Oracle, ServiceNow, Splunk, IBM, ]","[business analyst, data scientist, product man..."
2,/kaggle/input/resume4/R8.pdf,Madhuri Gogula\n,{516-358-1998},{mike@sqldatasolutionsinc.com},Bachelor's degree,"{2016, 2017, 1998, 2005, 2008, 2010, 2011, 201...","{Madison, Phoenix, McLean, Pittsburgh, Hyderab...","[Java, SQL, JavaScript, Spring Boot]",[],"[Oracle, IBM, American Express, Intel, Freddie...",[java developer]
3,/kaggle/input/resume4/R9.pdf,Technical Project Manager,"{646-361-0131, 646-361-1031}",{JNandini1324@gmail.com},Master's degree,"{2016, 2017, 2007, 2009, 2010, 2011, 2012, 201...",{},"[Java, SQL]",[],"[Oracle, Cisco Systems, Tech Mahindra, Symphon...","[business analyst, client delivery manager, cl..."
4,/kaggle/input/resume4/R10.pdf,Rama Krishna\nFull Stack Java Developer\n,{(516) 308-1026},{ramglobal5111@gmail.com},Bachelor's degree,"{2016, 2009, 2012, 2013, 2014, 2015}","{Portland, Allen, Mongo, Spring, Durham}","[Java, SQL, JavaScript, MongoDB, Spring Boot, ...",[Degree\nKoneru Lakshmaiah College],"[Oracle, ]","[data management, java developer, software dev..."


## 4 - Compare Testing Results

Get all the resumes in pdf format and perform preprocessing, and finally save it into dataframe.

In [18]:
# before preprocessing 
df

,file_name,name,phone_num,email,qualifications,graduated_year,location,skills,university,company,designation
0,/kaggle/input/resume4/R6.pdf,Gopi Mobile,{732-630-7623},{gopijavafullstack@gmail.com},Bachelor's degree,"{2016, 2017, 2000, 2010, 2011, 2012, 2014, 2015}","{Mobile, Los Angeles, Frisco, Ajax, Mongo, Hyd...","[Java, SQL, JavaScript, MongoDB, Spring Boot, ...",[],"[State Street, Oracle, Amazon, IBM, Pool, NCC,...","[business objects, data management, full stack..."
1,/kaggle/input/resume4/R7.pdf,Vora Phone,{201-532-6397},{kashyapkvora@gmail.com},Master's degree,"{2016, 2006, 2008, 2009, 2011, 2013, 2014, 2015}","{Baltimore, Mumbai, University, Smithfield, Wa...","[Python, Java, SQL, JavaScript]",[Johns Hopkins University],"[HP, Oracle, ServiceNow, Splunk, IBM, ]","[business analyst, data scientist, product man..."
2,/kaggle/input/resume4/R8.pdf,Madhuri Gogula\n,{516-358-1998},{mike@sqldatasolutionsinc.com},Bachelor's degree,"{2016, 2017, 1998, 2005, 2008, 2010, 2011, 201...","{Madison, Phoenix, McLean, Pittsburgh, Hyderab...","[Java, SQL, JavaScript, Spring Boot]",[],"[Oracle, IBM, American Express, Intel, Freddie...",[java developer]
3,/kaggle/input/resume4/R9.pdf,Technical Project Manager,"{646-361-0131, 646-361-1031}",{JNandini1324@gmail.com},Master's degree,"{2016, 2017, 2007, 2009, 2010, 2011, 2012, 201...",{},"[Java, SQL]",[],"[Oracle, Cisco Systems, Tech Mahindra, Symphon...","[business analyst, client delivery manager, cl..."
4,/kaggle/input/resume4/R10.pdf,Rama Krishna\nFull Stack Java Developer\n,{(516) 308-1026},{ramglobal5111@gmail.com},Bachelor's degree,"{2016, 2009, 2012, 2013, 2014, 2015}","{Portland, Allen, Mongo, Spring, Durham}","[Java, SQL, JavaScript, MongoDB, Spring Boot, ...",[Degree\nKoneru Lakshmaiah College],"[Oracle, ]","[data management, java developer, software dev..."


In [19]:
# after preprocessing
df2

,file_name,name,phone_num,email,qualifications,graduated_year,location,skills,university,company,designation
0,/kaggle/input/resume4/R6.pdf,Gopi Mobile,{732-630-7623},{gopijavafullstack@gmail.com},Bachelor's degree,"{2016, 2017, 2000, 2010, 2011, 2012, 2014, 2015}","{Mobile, Los Angeles, Frisco, Ajax, Mongo, Hyd...","[Java, SQL, JavaScript, MongoDB, Spring Boot, ...",[],"[State Street, Oracle, Amazon, IBM, Pool, NCC,...","[business objects, data management, full stack..."
1,/kaggle/input/resume4/R7.pdf,Vora Phone,{201-532-6397},{kashyapkvora@gmail.com},Master's degree,"{2016, 2006, 2008, 2009, 2011, 2013, 2014, 2015}","{Baltimore, Mumbai, University, Smithfield, Wa...","[Python, Java, SQL, JavaScript]",[Johns Hopkins University],"[HP, Oracle, ServiceNow, Splunk, IBM, ]","[business analyst, data scientist, product man..."
2,/kaggle/input/resume4/R8.pdf,Madhuri Gogula\n,{516-358-1998},{mike@sqldatasolutionsinc.com},Bachelor's degree,"{2016, 2017, 1998, 2005, 2008, 2010, 2011, 201...","{Madison, Phoenix, McLean, Pittsburgh, Hyderab...","[Java, SQL, JavaScript, Spring Boot]",[],"[Oracle, IBM, American Express, Intel, Freddie...",[java developer]
3,/kaggle/input/resume4/R9.pdf,Technical Project Manager,"{646-361-0131, 646-361-1031}",{JNandini1324@gmail.com},Master's degree,"{2016, 2017, 2007, 2009, 2010, 2011, 2012, 201...",{},"[Java, SQL]",[],"[Oracle, Cisco Systems, Tech Mahindra, Symphon...","[business analyst, client delivery manager, cl..."
4,/kaggle/input/resume4/R10.pdf,Rama Krishna\nFull Stack Java Developer\n,{(516) 308-1026},{ramglobal5111@gmail.com},Bachelor's degree,"{2016, 2009, 2012, 2013, 2014, 2015}","{Portland, Allen, Mongo, Spring, Durham}","[Java, SQL, JavaScript, MongoDB, Spring Boot, ...",[Degree\nKoneru Lakshmaiah College],"[Oracle, ]","[data management, java developer, software dev..."


In [28]:
# Hypothetical ground truth for test resumes (R6.pdf to R10.pdf)
# Replace this with actual ground truth data
ground_truth_data = {
    'file_name': [f"/kaggle/input/resume4/R{i}.pdf" for i in range(6, 11)],
    'name': ['John Doe', 'Jane Smith', 'Alice Johnson', 'Bob Brown', 'Charlie Davis'],
    'phone_num': [{'(123) 456-7890'}, {'(234) 567-8901'}, {'(345) 678-9012'}, {'(456) 789-0123'}, {'(567) 890-1234'}],
    'email': [{'john.doe@example.com'}, {'jane.smith@example.com'}, {'alice.j@example.com'}, {'bob.brown@example.com'}, {'charlie.davis@example.com'}],
    'qualifications': ["Bachelor's degree", "Master's degree", "PhD", "Bachelor's degree", "Diploma"],
    'graduated_year': [{2020}, {2018}, {2019}, {2021}, {2022}],
    'location': [{'New York'}, {'London'}, {'Paris'}, {'Tokyo'}, {'Sydney'}],
    'skills': [['Python', 'Java'], ['SQL', 'R'], ['Machine Learning', 'Python'], ['JavaScript', 'HTML'], ['CSS', 'Python']],
    'university': [['Harvard University'], ['Oxford University'], ['Sorbonne University'], ['Tokyo University'], ['Sydney University']],
    'company': [['Google'], ['Microsoft'], ['Amazon'], ['Facebook'], ['Apple']],
    'designation': [['Software Engineer'], ['Data Scientist'], ['Machine Learning Engineer'], ['Web Developer'], ['Frontend Developer']]
}
ground_truth_df = pd.DataFrame(ground_truth_data)

# Function to calculate performance metrics
def calculate_performance_metrics(extracted_df, ground_truth_df):
    metrics = {
        'Entity': [],
        'Precision': [],
        'Recall': [],
        'F1 Score': []
    }
    total_correct = 0
    total_entities = 0

    entity_types = ['name', 'phone_num', 'email', 'qualifications', 'graduated_year', 'location', 'skills', 'university', 'company', 'designation']

    for entity_type in entity_types:
        true_positives = 0
        false_positives = 0
        false_negatives = 0

        for i in range(len(extracted_df)):
            extracted = extracted_df.iloc[i][entity_type]
            ground_truth = ground_truth_df.iloc[i][entity_type]

            # Handle different types of entities
            if entity_type in ['phone_num', 'email', 'graduated_year', 'location']:
                # For sets (e.g., phone_num, email)
                extracted = extracted if isinstance(extracted, set) else set() if extracted is None else {extracted}
                ground_truth = ground_truth if isinstance(ground_truth, set) else set() if ground_truth is None else {ground_truth}
                tp = len(extracted & ground_truth)
                fp = len(extracted - ground_truth)
                fn = len(ground_truth - extracted)
            elif entity_type in ['skills', 'university', 'company', 'designation']:
                # For lists (e.g., skills, company)
                extracted = extracted if isinstance(extracted, list) else [] if extracted is None else [extracted]
                ground_truth = ground_truth if isinstance(ground_truth, list) else [] if ground_truth is None else [ground_truth]
                tp = len(set(extracted) & set(ground_truth))
                fp = len(set(extracted) - set(ground_truth))
                fn = len(set(ground_truth) - set(extracted))
            else:
                # For strings (e.g., name, qualifications)
                extracted = str(extracted).strip() if extracted else ""
                ground_truth = str(ground_truth).strip() if ground_truth else ""
                tp = 1 if extracted == ground_truth else 0
                fp = 1 if extracted and extracted != ground_truth else 0
                fn = 1 if ground_truth and extracted != ground_truth else 0

            true_positives += tp
            false_positives += fp
            false_negatives += fn

            # For total accuracy
            if entity_type in ['phone_num', 'email', 'graduated_year', 'location']:
                total_correct += tp
                total_entities += len(ground_truth)
            elif entity_type in ['skills', 'university', 'company', 'designation']:
                total_correct += tp
                total_entities += len(ground_truth)
            else:
                total_correct += tp
                total_entities += 1 if ground_truth else 0

        # Calculate precision, recall, F1
        precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        metrics['Entity'].append(entity_type)
        metrics['Precision'].append(precision)
        metrics['Recall'].append(recall)
        metrics['F1 Score'].append(f1)

    # Calculate total accuracy
    total_accuracy = total_correct / total_entities if total_entities > 0 else 0

    # Create metrics DataFrame
    metrics_df = pd.DataFrame(metrics)
    print("\nPerformance Metrics Report")
    print("------------------------------")
    print(metrics_df)
    print("\nTotal Accuracy:", total_accuracy)

# Calculate metrics for df (without preprocessing)
print("\nMetrics for Test Resumes (Without Preprocessing)")
calculate_performance_metrics(df, ground_truth_df)

# Calculate metrics for df2 (with preprocessing)
print("\nMetrics for Test Resumes (With Preprocessing)")
calculate_performance_metrics(df2, ground_truth_df)


Metrics for Test Resumes (Without Preprocessing)

Performance Metrics Report
------------------------------
           Entity  Precision  Recall  F1 Score
0            name          0       0         0
1       phone_num          0       0         0
2           email          0       0         0
3  qualifications          0       0         0
4  graduated_year          0       0         0
5        location          0       0         0
6          skills          0       0         0
7      university          0       0         0
8         company          0       0         0
9     designation          0       0         0

Total Accuracy: 0

Metrics for Test Resumes (With Preprocessing)

Performance Metrics Report
------------------------------
           Entity  Precision  Recall  F1 Score
0            name   0.000000     0.0  0.000000
1       phone_num   0.000000     0.0  0.000000
2           email   0.000000     0.0  0.000000
3  qualifications   0.400000     0.4  0.400000
4  graduated_y